In [110]:
import math

class Node:
    def __init__(self, key, x, y):
        self._key = key
        self._x = x
        self._y = y
        self._parent = None
        self._cluster = None
        
    @property
    def key(self):
        return self._key

    @property
    def x(self):
        return self._x
        
    @property
    def y(self):
        return self._y
    
    @property
    def parent(self):
        return self._parent
    
    @parent.setter
    def parent(self, node):
        self._parent = node
        
    @property
    def cluster(self):
        return self.find()._cluster
    
    @cluster.setter
    def cluster(self, cluster):
        self._cluster = cluster
        
    def distanceTo(self, node):
        return math.sqrt(math.pow(node.x - self.x, 2) + math.pow(node.y - self.y, 2))
    
    def find(self):
        if self == self.parent:
            return self
        else:
            return self.parent.find()
        
    def isInTheSameClusterWith(self, node):
        return self.cluster == node.cluster
        
    def __repr__(self):
        return '[Node key={}, loc=({}, {})]'.format(self._key, self._x, self._y)
    
# Unit Test
n = Node('0', 1, 2)
assert n.x == 1
assert n.y == 2
assert n.__repr__() == '[Node key=0, loc=(1, 2)]'

In [111]:
from random import randint

class Space:
    def __init__(self, minX, maxX, minY, maxY):
        self._minX = minX
        self._maxX = maxX
        self._minY = minY
        self._maxY = maxY
        self._nodes = []
        
    def generate(self, n):
        for i in range(0, n):
            self._nodes.append(Node(i, randint(self._minX, self._maxX), randint(self._minY, self._maxY)))
            
    @property
    def nodes(self):
        return self._nodes
            
    def __repr__(self):
        return self._nodes.__repr__()
    
# Unit Test
s = Space(0, 100, 0, 100)
s.generate(10)
assert len(s.nodes) == 10

In [112]:
class Cluster:
    def __init__(self, key, root):
        self._key = key
        self._root = root
        self._rank = 0
    
    @property
    def key(self):
        return self._key
    
    @key.setter
    def key(self, key):
        self._key = key
        
    @property
    def root(self):
        return self._root
    
    @root.setter
    def root(self, root):
        self._root = root
        
    @property
    def rank(self):
        return self._rank
    
    @rank.setter
    def rank(self, rank):
        self._rank = rank
        
    def union(self, s):
        if self.rank < s.rank:
            s.root.parent = self.root
            s.root = self.root
            self.rank = s.rank + 1
            return self
        else:
            self.root.parent = s.root
            self.root = s.root
            if self.rank == s.rank:
                s.rank += 1
            return s
                
    def __repr__(self):
        return '[Set key={}, root={}, rank={}]'.format(self._key, self._root, self._rank)

In [117]:
class Partition:
    def __init__(self, nodes):
        self._clusters = {}
        self._vertices = nodes
        self._edges = []
        for i in range(0, len(nodes)):
            for j in range(i + 1, len(nodes)):
                self._edges.append((nodes[i].distanceTo(nodes[j]), nodes[i], nodes[j]))
        
    def generateSet(self, key, root):
        root.parent = root
        c = Cluster(key, root)
        self._clusters[key] = c
        root.cluster = c
        return c
        
    def unionSet(self, setA, setB):
        c = setA.union(setB)
        
        if c.key == setA.key:
            if setB.key in self._clusters:
                del self._clusters[setB.key]
            setB.root.cluster = setA
        else:
            if setA.key in self._clusters:
                del self._clusters[setA.key]
            setA.root.cluster = setB
        return c
        
    def findSet(self, node):
        return node.find().cluster
    
    def cluster(self):
        edges = sorted(self._edges, key=lambda edge: edge[0])
        
        for i in range(0, len(self._vertices)):
            self.generateSet(i, self._vertices[i])
           
        for i in range(0, len(edges)):
            w, u, v = edges[i]
            if self.findSet(u) != self.findSet(v):
                self.unionSet(u.cluster, v.cluster)
                
        print(self._clusters)

         
s = Space(0, 100, 0, 100)
s.generate(5)
p = Partition(s.nodes)
p.cluster()

{2: [Set key=2, root=[Node key=2, loc=(41, 51)], rank=0]}
